# Rest API

### Create RestApi Server

In [2]:
import json
import cherrypy
import redis

REDIS_HOST = 'redis-11747.c300.eu-central-1-1.ec2.cloud.redislabs.com'
REDIS_PORT = 11747
REDIS_PASSWORD = 'i4Wds8wGEcQ5PVQwBMmEQL17GH4XdXKH'

client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD)
is_connected = client.ping()
print('Redis client is connected ', is_connected)


class Devices(object):
    exposed = True

    @cherrypy.expose
    def GET(self):
        c = client.keys('*')
        devices = {'mac_addresses': list(set(map(lambda x: x.decode('utf-8').split(':')[0], client.keys('*'))))}
        return json.dumps(devices)


class Device(object):
    exposed = True

    @cherrypy.expose
    def GET(self, mac_address=None, **query_params):
        if mac_address:
            addresses = set(map(lambda x: x.decode('utf-8').split(':')[0], client.keys('*')))
            if mac_address not in addresses:
                raise cherrypy.HTTPError(404, 'Invalid MAC address')

            if 'from' in query_params and 'to' in query_params:
                battery_infos = client.ts().range(mac_address + ':battery', query_params['from'], query_params['to'])
                power_infos = client.ts().range(mac_address + ':power', query_params['from'], query_params['to'])

                timestamps = list(map(lambda x: x[0], battery_infos))
                battery_infos = list(map(lambda x: [1][0], battery_infos))
                power_infos = list(map(lambda x: [1][0], power_infos))

                device = {
                    'mac_address': mac_address,
                    'timestamps': timestamps,
                    'battery_levels': battery_infos,
                    'power_plugged': power_infos
                }

                return str(device)
            elif 'from' not in query_params:
                raise cherrypy.HTTPError(400, 'Missing start time')
            elif 'to' not in query_params:
                raise cherrypy.HTTPError(400, 'Missing end time')
        else:
            raise cherrypy.HTTPError(400, 'Missing MAC address')

    @cherrypy.expose
    def DELETE(self, mac_address=None):
        if mac_address:
            addresses = set(map(lambda x: x.decode('utf-8').split(':')[0], client.keys('*')))
            if mac_address not in addresses:
                raise cherrypy.HTTPError(404, 'Invalid MAC address')

            client.delete(mac_address + ':battery')
            client.delete(mac_address + ':plugged')
            client.delete(mac_address + ':power')
        else:
            raise cherrypy.HTTPError(400, 'Missing MAC address')
        return "DELETED"


if __name__ == '__main__':
    conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}}

    cherrypy.tree.mount(Device(), '/device', conf)
    cherrypy.tree.mount(Devices(), '/devices', conf)

    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080})

    cherrypy.engine.start()
    cherrypy.engine.block()
    

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=82f6f9dd-e2eb-4faf-a867-1a0a5dea3d6b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>